In [1]:
import sys
from pathlib import Path

IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules
if IS_KAGGLE:
    repo_path = Path("../input/crypto-prediction")
elif IS_COLAB:
    from google.colab import drive

    drive.mount("/content/gdrive")
    repo_path = Path("/content/gdrive/MyDrive/crypto-prediction")
else:
    repo_path = Path("/home/matias/crypto-prediction")
sys.path.append(str(repo_path))

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import tensorflow as tf
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchviz import make_dot

import get_data
import wandb
from tools import dataframe_reformat, inspect_code, plotting, training, wandb_api

log_wandb = True
tf.config.list_physical_devices("CPU")[0].device_type


2022-03-16 15:36:01.749037: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-16 15:36:01.749081: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-03-16 15:36:04.056284: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-16 15:36:04.056319: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-16 15:36:04.056332: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Matias): /proc/driver/nvidia/version does not exist


'CPU'

In [2]:
if log_wandb:
    import wandb

    wandb_api.login()
    run = wandb.init(
        project="crypto-prediction",
        group="Initial Gan",
        job_type="test",
    )


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: matiasetcheverry (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/matias/.netrc
2022-03-16 15:36:06.234180: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-16 15:36:06.234221: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
if log_wandb:
    config = wandb.config
else:
    config = {}

config["job_type"] = run.job_type if "run" in locals() else "test"
config["log_wandb"] = False
config["train_test_split"] = 0.7
config["nb_previous_close"] = 2
config["batch_size"] = 32
config["n_discriminator"] = 50
config["gp_weight"] = 10
config["learning_rate_generator"] = 0.0001
config["learning_rate_discriminator"] = 0.0004
config["beta1"] = 0.5
config["beta2"] = 0.9


In [4]:
class DataModule:
    def __init__(
        self,
        config,
        csv_file=None,
        train_df=None,
        test_df=None,
        train_dataset=None,
        validation_dataset=None,
    ):
        super().__init__()
        self.config = config

        if csv_file is not None:
            self.df = pd.read_csv(csv_file, delimiter=";")
            self.df["BEGINNING_DATE"] = pd.to_datetime(
                self.df["BEGINNING_DATE"], dayfirst=True
            )
            self.df["ENDING_DATE"] = pd.to_datetime(
                self.df["ENDING_DATE"], dayfirst=True
            )
            self.df["TICKER"] += "-USD"

        self.train_df = train_df.convert_dtypes() if train_df is not None else None
        self.test_df = test_df.convert_dtypes() if test_df is not None else None
        self.train_dataset = train_dataset
        self.validation_dataset = validation_dataset

    def _preprocess_klines(
        self,
        data=None,
        ticker=None,
        beginning_date=None,
        ending_date=None,
        interval="1d",
    ):
        if data is None:
            data = get_data.select_data(
                ticker,
                interval,
                beginning_date=beginning_date,
                ending_date=ending_date,
            )
        data.dropna(axis=0, inplace=True)
        data.drop(labels="Date", axis=1, inplace=True)
        data.replace(
            to_replace=[np.inf, -np.inf, np.float64("inf"), -np.float64("inf")],
            value=0,
            inplace=True,
        )
        self.data = data
        idx_close = list(data.columns).index("Close")
        scaler = MinMaxScaler(feature_range=(0, 1))
        klines = tf.convert_to_tensor(scaler.fit_transform(data))

        data_close = klines[:, idx_close]
        single_close = tf.convert_to_tensor(
            data_close[self.config["nb_previous_close"] :]
        )
        multiple_close = tf.stack(
            [
                data_close[i : i + self.config["nb_previous_close"]]
                for i in range(len(data_close) - self.config["nb_previous_close"])
            ]
        )
        multiple_klines = tf.stack(
            [
                klines[i : i + self.config["nb_previous_close"], :]
                for i in range(len(klines) - self.config["nb_previous_close"])
            ]
        )
        return multiple_klines, single_close, multiple_close

    def prepare_data(self):
        for _, row in self.df.iterrows():
            _ = get_data.select_data(
                row["TICKER"],
                "1d",
                beginning_date=row["BEGINNING_DATE"],
                ending_date=row["ENDING_DATE"],
            )

    def setup(self):
        klines = []
        single_closes = []
        multiple_closes = []
        for _, row in self.df.iterrows():
            kline, single_close, multiple_close = self._preprocess_klines(
                ticker=row["TICKER"],
                beginning_date=row["BEGINNING_DATE"],
                ending_date=row["ENDING_DATE"],
            )
            klines.append(kline)
            single_closes.append(single_close)
            multiple_closes.append(multiple_close)

        klines = tf.data.Dataset.from_tensor_slices(
            tf.cast(tf.concat(klines, axis=0), dtype=tf.float32)
        )
        single_closes = tf.data.Dataset.from_tensor_slices(
            tf.cast(
                tf.expand_dims(
                    (tf.expand_dims(tf.concat(single_closes, axis=0), axis=-1)), axis=-1
                ), 
                dtype=tf.float32)
            
        )
        multiple_closes = tf.data.Dataset.from_tensor_slices(
            tf.cast(tf.expand_dims(tf.concat(multiple_closes, axis=0), axis=-1), dtype=tf.float32,),
            
        )
        dataset = (
            tf.data.Dataset.zip((klines, multiple_closes, single_closes))
            .shuffle(len(klines), reshuffle_each_iteration=True)
            .batch(
                self.config["batch_size"],
                drop_remainder=False,
                num_parallel_calls=tf.data.AUTOTUNE,
            )
        )

        train_size = int(config["train_test_split"] * len(dataset))
        self.train_dataset = dataset.take(train_size)
        self.val_dataset = dataset.skip(train_size)


dm = DataModule(config, "DATE.csv")
dm.prepare_data()
dm.setup()
print(len(dm.train_dataset), len(dm.val_dataset))
train_dataset = dm.train_dataset
print(
    next(iter(train_dataset))[0].shape,
    next(iter(train_dataset))[1].shape,
    next(iter(train_dataset))[2].shape,
)
print(dm.train_dataset)


2022-03-16 15:36:08.383142: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


264 114
(32, 2, 29) (32, 2, 1) (32, 1, 1)
<TakeDataset element_spec=(TensorSpec(shape=(None, 2, 29), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))>


In [5]:
from __future__ import division, print_function

import sys
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import (
    LSTM,
    Bidirectional,
    Concatenate,
    Conv1D,
    Conv2D,
    Dense,
    Dropout,
    Input,
    LeakyReLU,
    ReLU,
    Reshape,
    Layer,
)
from tensorflow.keras.activations import sigmoid
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.models import Model


class WGANGP(Model):
    def __init__(self, config):
        super().__init__()
        self.config = config
        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5

        # Build the generator and critic
        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()

        generator_optimizer = tf.keras.optimizers.Adam(
            learning_rate=self.config["learning_rate_generator"],
            beta_1=self.config["beta1"],
            beta_2=self.config["beta2"],
        )
        discriminator_optimizer = tf.keras.optimizers.Adam(
            learning_rate=self.config["learning_rate_discriminator"],
            beta_1=self.config["beta1"],
            beta_2=self.config["beta2"],
        )

        self.compile(
            d_optimizer=discriminator_optimizer,
            g_optimizer=generator_optimizer,
            g_loss_fn=self.generator_loss,
            d_loss_fn=self.discriminator_loss,
        )

        # # -------------------------------
        # # Construct Computational Graph
        # #       for the Discriminator
        # # -------------------------------

        # # Freeze generator's layers while training discriminator
        # self.generator.trainable = False

        # # Image input (real sample)
        # klines = Input(shape=(self.config["nb_previous_close"], 29))

        # # Generate image based of noise (fake sample)
        # fake_close = self.generator(klines)
        # real_close = Input(shape=(1, 1))
        # multiple_closes = Input(shape=(self.config["nb_previous_close"], 1))
        # fake_closes = Concatenate(axis=1)([multiple_closes, fake_close])
        # real_closes = Concatenate(axis=1)([multiple_closes, real_close])

        # # Discriminator determines validity of the real and fake images
        # fake = self.discriminator(fake_closes)
        # valid = self.discriminator(real_closes)

        # # Construct weighted average between real and fake images
        # interpolated_closes = RandomWeightedAverage()([fake_closes, real_closes])
        # # Determine validity of weighted sample
        # validity_interpolated = self.discriminator(interpolated_closes)

        # # Use Python partial to provide loss function with additional
        # # 'averaged_samples' argument
        # partial_gp_loss = partial(
        #     self.gradient_penalty_loss, averaged_samples=interpolated_closes
        # )
        # partial_gp_loss.__name__ = "gradient_penalty"  # Keras requires function names

        # self.critic_model = Model(
        #     inputs=[real_img, z_disc], outputs=[valid, fake, validity_interpolated]
        # )
        # self.critic_model.compile(
        #     loss=[self.wasserstein_loss, self.wasserstein_loss, partial_gp_loss],
        #     optimizer=optimizer,
        #     loss_weights=[1, 1, 10],
        # )
        # # -------------------------------
        # # Construct Computational Graph
        # #         for Generator
        # # -------------------------------

        # # For the generator we freeze the critic's layers
        # self.critic.trainable = False
        # self.generator.trainable = True

        # # Sampled noise for input to generator
        # z_gen = Input(shape=(self.latent_dim,))
        # # Generate images based of noise
        # img = self.generator(z_gen)
        # # Discriminator determines validity
        # valid = self.critic(img)
        # # Defines generator model
        # self.generator_model = Model(z_gen, valid)
        # self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)

    def build_generator(self):
        inputs = Input(
            shape=(self.config["nb_previous_close"], 29),
            batch_size=self.config["batch_size"],
        )
        outputs = Conv1D(32, 2, activation=LeakyReLU(alpha=0.1))(inputs)
        outputs = Bidirectional(LSTM(64, dropout=0.3, activation=ReLU()))(outputs)
        outputs = Dense(64, activation=LeakyReLU(alpha=0.1))(outputs)
        outputs = Dropout(0.2)(outputs)
        outputs = Dense(32, activation=LeakyReLU(alpha=0.1))(outputs)
        outputs = Dropout(0.2)(outputs)
        outputs = Dense(1, activation=None)(outputs)
        outputs = Reshape(target_shape=(1, 1))(outputs)
        generator = Model(inputs=inputs, outputs=outputs, name="genrator")
        return generator

    def build_discriminator(self):
        inputs = Input(
            shape=(self.config["nb_previous_close"] + 1, 1),
            batch_size=self.config["batch_size"],
        )
        outputs = Conv1D(32, 2, activation=LeakyReLU(alpha=0.1))(inputs)
        outputs = Conv1D(64, 2, activation=LeakyReLU(alpha=0.1))(outputs)
        outputs = Dense(64, activation=LeakyReLU(alpha=0.1))(outputs)
        outputs = Dropout(0.2)(outputs)
        outputs = Dense(32, activation=LeakyReLU(alpha=0.1))(outputs)
        outputs = Dropout(0.2)(outputs)
        outputs = Dense(1, activation=None)(outputs)
        discriminator = Model(inputs=inputs, outputs=outputs, name="discriminator")
        return discriminator

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    def discriminator_loss(self, real, fake, current_metrics={}):
        real_loss = tf.reduce_mean(real)
        fake_loss = tf.reduce_mean(fake)

        current_metrics["real_discriminator"] = real_loss
        current_metrics["fake_discriminator"] = fake_loss
        return fake_loss - real_loss

    # Define the loss functions for the generator.
    def generator_loss(self, fake, real, current_metrics={}):
        fake_loss = -tf.reduce_mean(fake)
        far_loss = tf.keras.metrics.mean_squared_error(tf.squeeze(real), tf.squeeze(fake))
        g_loss = fake_loss + 0.5 * far_loss
        current_metrics["fake_generator"] = fake_loss
        current_metrics["close_generator"] = far_loss
        current_metrics["g_loss"] = g_loss
        return fake_loss

    def gradient_penalty(self, real_images, fake_images, current_metrics={}):
        """Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        alpha = tf.random.normal((self.config["batch_size"], 1), 0.0, 1.0)
        diff = fake_images - real_images

        interpolated = real_images + tf.multiply(diff, alpha[:, tf.newaxis])

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = self.discriminator(interpolated, training=True)

        grads = gp_tape.gradient(pred, [interpolated])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)

        current_metrics["penalty_discriminator"] = gp
        return gp

    def update_metrics(self, metrics, current_metrics, reduction=None):
        if reduction == "mean":
            factor_reduction = self.config["n_discriminator"]
        else:
            factor_reduction = 1

        for name, value in current_metrics.items():
            metrics[name] = metrics.get(name, 0) + value / factor_reduction

    def train_step(self, data):
        klines, previous_closes, real_close = data

        metrics = {
            "fake_discriminator": 0,
            "real_discriminator": 0,
            "penalty_discriminator": 0,
            "d_loss": 0,
            "fake_generator": 0,
            "close_generator": 0,
            "g_loss": 0
        }
        for _ in range(self.config["n_discriminator"]):
            current_metrics = {}
            with tf.GradientTape() as tape:
                fake_close = self.generator(klines, training=True)
                fake_closes = tf.concat(
                    [previous_closes, fake_close],
                    axis=1,
                )
                real_closes = tf.concat([previous_closes, real_close], axis=1)
                fake = self.discriminator(fake_closes, training=True)
                real = self.discriminator(real_closes, training=True)
                d_cost = self.d_loss_fn(real, fake, current_metrics)
                gp = self.gradient_penalty(real_closes, fake_closes, current_metrics)
                d_loss = d_cost + gp * self.config["gp_weight"]
                current_metrics["d_loss"] = d_loss

            self.update_metrics(metrics, current_metrics, reduction="mean")

            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            self.d_optimizer.apply_gradients(
                zip(d_gradient, self.discriminator.trainable_variables)
            )

        with tf.GradientTape() as tape:
            current_metrics = {}
            fake_close = self.generator(klines, training=True)
            fake_closes = tf.concat(
                [previous_closes, fake_close],
                axis=1,
            )
            fake = self.discriminator(fake_closes, training=True)
            g_loss = self.g_loss_fn(fake, real, current_metrics)
        self.update_metrics(metrics, current_metrics)
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )
        return metrics

config["gp_weight"] = 10
config["n_discriminator"] = 5
wgan = WGANGP(config)

# Instantiate the optimizer for both networks
# (learning_rate=0.0002, beta_1=0.5 are recommended

# Define the loss functions for the discriminator,
# which should be (fake_loss - real_loss).
# We will add the gradient penalty later to this loss function.


# Set the number of epochs for trainining.
epochs = 300


# Start training the model.
wgan.fit(train_dataset, epochs=epochs, callbacks=[WandbCallback()])


Epoch 1/300
264/264 [==============================] - 15s 16ms/step - fake_discriminator: -0.0766 - real_discriminator: -0.0275 - penalty_discriminator: 0.0416 - d_loss: 0.3667 - fake_generator: 0.0779 - close_generator: 0.0229 - g_loss: 0.0893 - _timestamp: 1647441386.0000 - _runtime: 22.0000
Epoch 2/300
264/264 [==============================] - 5s 18ms/step - fake_discriminator: -0.0694 - real_discriminator: -0.0551 - penalty_discriminator: 0.0159 - d_loss: 0.1445 - fake_generator: 0.0674 - close_generator: 0.0196 - g_loss: 0.0772 - _timestamp: 1647441391.0000 - _runtime: 27.0000
Epoch 3/300
264/264 [==============================] - 4s 15ms/step - fake_discriminator: -0.1045 - real_discriminator: -0.0934 - penalty_discriminator: 0.0147 - d_loss: 0.1361 - fake_generator: 0.1061 - close_generator: 0.0505 - g_loss: 0.1314 - _timestamp: 1647441395.0000 - _runtime: 31.0000
Epoch 4/300
264/264 [==============================] - 4s 16ms/step - fake_discriminator: -0.0201 - real_discrimin

In [ ]:
run.finish()

close_generator,▁▂▁▁▂▃▃▅▇█
d_loss,▅▅▄▅█▇▁▂▁▃
epoch,▁▂▃▃▄▅▆▆▇█
fake_discriminator,▃▆▄▅▅▆██▂▁
fake_generator,▆▃▄▄▅▃▁▂▇█
g_loss,▅▃▄▃▅▃▁▂▇█
penalty_discriminator,▄▆▄▄█▆▁▂▂▃
real_discriminator,▃▆▅▅▄▆██▃▁
close_generator,0.1061
d_loss,0.11471
epoch,9


In [ ]:
class Generator(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.conv_layer = nn.Sequential(
            nn.Conv1d(config["nb_previous_close"], 32, kernel_size=2),
            nn.LeakyReLU(),
        )
        self.lstm_layer = nn.LSTM(
            28, 64, num_layers=1, batch_first=True, bidirectional=True, dropout=0.3
        )
        self.fc_layers = nn.Sequential(
            nn.ReLU(),
            # nn.Flatten(),
            nn.Linear(in_features=2 * 64, out_features=64),
            nn.LeakyReLU(),
            # nn.Dropout2d(0.2),
            nn.Linear(in_features=64, out_features=32),
            nn.LeakyReLU(),
            # nn.Dropout2d(0.2),
            nn.Linear(in_features=32, out_features=1),
        )

    def forward(self, x):
        x = self.conv_layer(x)
        output, (hidden_state, cell_state) = self.lstm_layer(x)
        hidden_state = torch.permute(hidden_state, (1, 0, 2)).reshape(-1, 2 * 64)
        x = self.fc_layers(hidden_state)
        return x


class Discriminator(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.conv_layer = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=2),
            nn.LeakyReLU(),
            nn.Conv1d(32, 64, kernel_size=2),
            nn.LeakyReLU(),
        )
        _, nb_filters, width = self.conv_layer(torch.rand(1, 1, 21)).shape
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=nb_filters * width, out_features=64),
            nn.LeakyReLU(),
            # nn.Dropout2d(0.2),
            nn.Linear(in_features=64, out_features=32),
            nn.LeakyReLU(),
            # nn.Dropout2d(0.2),
            nn.Linear(in_features=32, out_features=1),
        )

    def forward(self, z):
        x = self.conv_layer(z.unsqueeze(1))
        x = self.fc_layers(x)
        return x


class GAN(pl.LightningModule):
    def __init__(
        self,
        config,
    ):
        super().__init__()
        self.config = config

        # networks
        self.generator = Generator(self.config)
        self.discriminator = Discriminator(self.config)
        self.automatic_optimization = False

    def forward(self, z):
        return self.generator(z)

    def adversarial_loss(self, y_hat, y):
        return nn.BCELoss()(y_hat, y)

    def generator_loss(self, fake, y_hat, y):
        fake_generator = torch.mean(fake)
        close_generator = torchmetrics.MeanSquaredError()(y_hat, y)
        sign_generator = torch.mean(torch.abs(torch.sign(y_hat) - torch.sign(y)))
        return fake_generator, close_generator, sign_generator

    def discriminator_loss(self, real, fake, y_hat, y):
        fake_discriminator = torch.mean(fake)
        real_discriminator = torch.mean(real)
        return fake_discriminator, real_discriminator

    def _training_step_generator(self, batch, opt=None):
        klines, multiple_close, y = batch
        fake_close = self(klines)
        fake = self.discriminator(torch.cat([multiple_close, fake_close], dim=1))
        fake_generator, close_generator, sign_generator = self.generator_loss(
            fake, fake_close, y
        )
        g_loss = -fake_generator + 0.5 * close_generator + 0.5 * sign_generator
        if opt is not None:
            opt.zero_grad()
            self.manual_backward(g_loss)
            opt.step()

        return {
            "g_loss": g_loss,
            "fake_generator": fake_generator,
            "close_generator": close_generator,
            "sign_generator": sign_generator,
        }

    def _training_step_discriminator(self, batch, opt=None, steps=5):
        klines, multiple_close, y = batch
        D_loss = 0
        Fake_discriminator = 0
        Real_discriminator = 0
        for _ in range(steps):
            fake_close = self(klines)
            fake = self.discriminator(torch.cat([multiple_close, fake_close], dim=1))
            real = self.discriminator(torch.cat([multiple_close, y], dim=1))
            fake_discriminator, real_discriminator = self.discriminator_loss(
                real, fake, fake_close, y
            )
            d_loss = fake_discriminator - real_discriminator
            if opt is not None:
                opt.zero_grad()
                self.manual_backward(d_loss)
                opt.step()

            D_loss += d_loss / steps
            Fake_discriminator += fake_discriminator / steps
            Real_discriminator += real_discriminator / steps

        return {
            "d_loss": D_loss,
            "fake_discriminator": Fake_discriminator,
            "real_discriminator": Real_discriminator,
        }

    def training_step(self, batch, batch_idx):
        opt_g, opt_d = self.optimizers()

        metrics = {}
        metrics.update(self._training_step_generator(batch, opt_g))
        metrics.update(self._training_step_discriminator(batch, opt_d, steps=1))

        self.log_dict(
            metrics,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
        )

    def validation_step(self, batch, batch_idx):
        metrics = {}
        metrics.update(self._training_step_generator(batch))
        metrics.update(self._training_step_discriminator(batch, steps=1))
        metrics = {
            "val_" + metric_name: metric_value
            for metric_name, metric_value in metrics.items()
        }

        self.log_dict(
            metrics,
            prog_bar=True,
            on_step=False,
            on_epoch=True,
        )
        return metrics

    def configure_optimizers(self):
        opt_g = torch.optim.Adam(
            self.generator.parameters(),
            lr=self.config["learning_rate_generator"],
            # betas=(self.config["beta1"], self.config["beta2"]),
        )
        opt_d = torch.optim.Adam(
            self.discriminator.parameters(),
            lr=self.config["learning_rate_discriminator"],
            betas=(self.config["beta1"], self.config["beta2"]),
        )
        return opt_g, opt_d


model = GAN(config)

model_checkpoint = pl.callbacks.model_checkpoint.ModelCheckpoint(
    dirpath=run.dir if "run" in locals() else "tmp/",
    filename="{epoch}-{val_loss:.3f}",
    monitor="_generatorg_loss",
    mode="min",
    verbose=True,
    save_last=True,
)

script_checkpoint = training.ScriptCheckpoint(
    dirpath=run.dir if "run" in locals() else "tmp/",
)

callbacks = [script_checkpoint]
log = None
if config["job_type"] == "train" or False:
    callbacks.append(model_checkpoint)
    print(f"[INFO]: saving models.")
else:
    print(f"[INFO]: not saving models.")
if config["job_type"] == "debug":
    log = "all"

if config["log_wandb"]:
    wandb_logger = pl.loggers.WandbLogger()
    wandb_logger.watch(model, log=log, log_graph=True)
else:
    wandb_logger = None
trainer = pl.Trainer(
    max_epochs=150,
    callbacks=callbacks,
    logger=wandb_logger,
    devices="auto",
    accelerator="auto",
    #     limit_train_batches=3,
    #     limit_val_batches=3,
)
trainer.fit(model, dm)


NameError: name 'nn' is not defined

In [ ]:
compteur = 0
for i in range(len(previous_x)):
    for j in range(len(previous_x[i])):
        if not torch.equal(previous_x[i][0], previous_x[i][j]):
            compteur += 1
print(compteur)


0


In [ ]:
compteur = 0
for i in range(len(preprevious_x)):
    for j in range(len(preprevious_x[i])):
        if not torch.equal(preprevious_x[i][0], preprevious_x[i][j]):
            compteur += 1
print(compteur)


0


In [ ]:
compteur = 0
for i in range(len(following_x)):
    for j in range(len(following_x[i])):
        if not torch.equal(following_x[i][0], following_x[i][j]):
            compteur += 1
print(compteur)


0


In [ ]:
batch = previous_x[-1]
klines1 = batch[14]
klines2 = batch[15]
print(klines1.shape)
print(
    model.generator.fc_layers(
        klines1.unsqueeze(0),
    )
)


IndexError: list index out of range